In [485]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import re
import numpy as np
import openai
from openai.error import ServiceUnavailableError, APIConnectionError, APIError, AuthenticationError, InvalidRequestError, RateLimitError, Timeout, TryAgain
import os
import time
import json 
import warnings


list_of_openai_errors = (ServiceUnavailableError, APIConnectionError, APIError, AuthenticationError, 
                         InvalidRequestError, RateLimitError, Timeout, TryAgain)

rate_limit_exceeded = False

%store -r exchange_rate_api_url
%store -r openai_key
%store -r azure_container_name
%store -r storage_account_name
#%store -r sas_connection_string

openai.api_key = openai_key
exchange_rate_api_url = exchange_rate_api_url
container_name = azure_container_name[0]
#connection_string = sas_connection_string


try:
    df = pd.read_csv("Jobberman_data_raw.csv")
    df = df.drop_duplicates()
    df.to_parquet("Jobberman_data_raw.parquet", index = False)
    #df.to_parquet("Jobberman_data_raw.parquet")
    #parquet_df = pd.read_parquet("Jobberman_data_raw.parquet")
    #parquet_df = pd.concat([parquet_df, df], ignore_index=True)
    
    
except FileNotFoundError as e:
    print(e)
    df = ""
    
try:
    remainder_df = pd.read_csv("Jobberman_data_raw_remainder.csv")
    
    
except FileNotFoundError as e:
    print(e)
    remainder_df = ""
    
else:
    remainder_df = remainder_df[['Job_Name', 'Company_Name', 'Industry', 'Location', 'Job_Type',
           'Job_Summary', 'Education_Requirements',
           'Experience_Level_Requirements', 'Experience_Length_Requirements',
           'Job_Description', 'Salary', 'Date_Posted', 'Scrape_Time', 'url']]
    
    remainder_df.rename(columns = {"Salary_Range": "Salary"}, inplace = True)
    
    
    if isinstance(df, str):
        df = remainder_df.copy()
    
    else:
        df = pd.concat([df, remainder_df])
        df = df.drop_duplicates(keep = False)
        
        
print("Raw parquet file created! The raw CSV file will be deleted at the end of this process")

[Errno 2] No such file or directory: 'Jobberman_data_raw.csv'
Raw parquet file created! The raw CSV file will be deleted at the end of this process


In [486]:
df

,Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Date_Posted,Scrape_Time,url
0,Driver,\n\nICST Solutions\n\n,\n\nEngineering & Technology\n\n,Abuja,Full Time,We seek to hire a suitable candidate to fill t...,High School (S.S.C.E),Mid level,4 years,Diploma/ SSCE or O’levels equivalentA minimum ...,"\nLess than 75,000\n",3 days ago,2023-11-13 05:51:49,https://www.jobberman.com/listings/driver-840j2p
1,Driver,\n\nICST Solutions\n\n,\n\nEngineering & Technology\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,High School (S.S.C.E),Mid level,4 years,Diploma/ SSCE or O’levels equivalentA minimum ...,"\nLess than 75,000\n",3 days ago,2023-11-13 05:51:59,https://www.jobberman.com/listings/driver-q2qmkz
2,Digital Marketer And Social Media Administrator,\n\nICST Solutions\n\n,\n\nMarketing & Communications\n\n,Abuja,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,3 years,Develop marketing strategies that align with t...,"\n150,000 - 250,000\n",3 days ago,2023-11-13 05:52:10,https://www.jobberman.com/listings/digital-mar...
3,Chief Information Security Officer,\n\nJubilee-Life Mortgage Bank\n\n,\n\nEngineering & Technology\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,8 years,"Responsible for\nidentifying, developing, impl...","\n400,000 - 600,000\n",3 days ago,2023-11-13 05:52:20,https://www.jobberman.com/listings/chief-infor...
4,Sales Officer (Network Foundation and Voice an...,\n\nICST Solutions\n\n,\n\nSales\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Senior level,4 years,To reach and exceed all quarterly and annual t...,"\n250,000 - 400,000\n",3 days ago,2023-11-13 05:52:31,https://www.jobberman.com/listings/sales-offic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Senior Software Engineer,\n\nAyulla Inc.\n\n,\n\nSoftware & Data\n\n,Remote (Work From Home),Full Time,We are looking for a talented Python/Django ba...,Degree,Senior level,5 years,"Develop well-designed, scalable, and robust so...","\n400,000 - 600,000\n",1 week ago,2023-11-13 06:01:42,https://www.jobberman.com/listings/senior-soft...
246,Marketing Executive,\n\nLifemate Nigeria Limited\n\n,\n\nMarketing & Communications\n\n,Abeokuta & Ogun State,Full Time,We are looking to hire a suitable candidate to...,OND,Entry level,1 year,Identify and meet with furniture dealers to es...,"\n75,000 - 150,000\n",1 week ago,2023-11-13 06:01:51,https://www.jobberman.com/listings/marketing-e...
247,Architect,\n\nLifemate Nigeria Limited\n\n,\n\nBuilding & Architecture\n\n,Lagos,Full Time,We are looking to hire a suitable candidate to...,Degree,Entry level,1 year,Manage the company’s corporate prototype desig...,"\n75,000 - 150,000\n",1 week ago,2023-11-13 06:02:00,https://www.jobberman.com/listings/architect-x...
248,React Native Developer,\n\nLeaders Network\n\n,\n\nSoftware & Data\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,2 years,Mobile App Development: Design and develop mob...,"\n150,000 - 250,000\n",1 week ago,2023-11-13 06:02:09,https://www.jobberman.com/listings/react-nativ...


In [487]:
df.drop_duplicates()

,Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Date_Posted,Scrape_Time,url
0,Driver,\n\nICST Solutions\n\n,\n\nEngineering & Technology\n\n,Abuja,Full Time,We seek to hire a suitable candidate to fill t...,High School (S.S.C.E),Mid level,4 years,Diploma/ SSCE or O’levels equivalentA minimum ...,"\nLess than 75,000\n",3 days ago,2023-11-13 05:51:49,https://www.jobberman.com/listings/driver-840j2p
1,Driver,\n\nICST Solutions\n\n,\n\nEngineering & Technology\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,High School (S.S.C.E),Mid level,4 years,Diploma/ SSCE or O’levels equivalentA minimum ...,"\nLess than 75,000\n",3 days ago,2023-11-13 05:51:59,https://www.jobberman.com/listings/driver-q2qmkz
2,Digital Marketer And Social Media Administrator,\n\nICST Solutions\n\n,\n\nMarketing & Communications\n\n,Abuja,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,3 years,Develop marketing strategies that align with t...,"\n150,000 - 250,000\n",3 days ago,2023-11-13 05:52:10,https://www.jobberman.com/listings/digital-mar...
3,Chief Information Security Officer,\n\nJubilee-Life Mortgage Bank\n\n,\n\nEngineering & Technology\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,8 years,"Responsible for\nidentifying, developing, impl...","\n400,000 - 600,000\n",3 days ago,2023-11-13 05:52:20,https://www.jobberman.com/listings/chief-infor...
4,Sales Officer (Network Foundation and Voice an...,\n\nICST Solutions\n\n,\n\nSales\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Senior level,4 years,To reach and exceed all quarterly and annual t...,"\n250,000 - 400,000\n",3 days ago,2023-11-13 05:52:31,https://www.jobberman.com/listings/sales-offic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Senior Software Engineer,\n\nAyulla Inc.\n\n,\n\nSoftware & Data\n\n,Remote (Work From Home),Full Time,We are looking for a talented Python/Django ba...,Degree,Senior level,5 years,"Develop well-designed, scalable, and robust so...","\n400,000 - 600,000\n",1 week ago,2023-11-13 06:01:42,https://www.jobberman.com/listings/senior-soft...
246,Marketing Executive,\n\nLifemate Nigeria Limited\n\n,\n\nMarketing & Communications\n\n,Abeokuta & Ogun State,Full Time,We are looking to hire a suitable candidate to...,OND,Entry level,1 year,Identify and meet with furniture dealers to es...,"\n75,000 - 150,000\n",1 week ago,2023-11-13 06:01:51,https://www.jobberman.com/listings/marketing-e...
247,Architect,\n\nLifemate Nigeria Limited\n\n,\n\nBuilding & Architecture\n\n,Lagos,Full Time,We are looking to hire a suitable candidate to...,Degree,Entry level,1 year,Manage the company’s corporate prototype desig...,"\n75,000 - 150,000\n",1 week ago,2023-11-13 06:02:00,https://www.jobberman.com/listings/architect-x...
248,React Native Developer,\n\nLeaders Network\n\n,\n\nSoftware & Data\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,2 years,Mobile App Development: Design and develop mob...,"\n150,000 - 250,000\n",1 week ago,2023-11-13 06:02:09,https://www.jobberman.com/listings/react-nativ...


In [488]:
def checking_recorded_column_names(dataframe):
    df = dataframe.copy()
    mask_check = all (df[col].isin(df.columns).sum() for col in df.columns)

    if mask_check:
        mask = df["Salary"].isin(df.columns)
        idx = df.loc[mask, :].index
        print(idx)
        df.drop(idx, inplace = True)
        print(f"{len(idx)} rows dropped because they are recorded column names")
        
    return df

In [489]:
def remove_new_line(dataframe):
    df = dataframe.copy()
    
    for col_name in df.columns:
        df[col_name] = df[col_name].str.strip("\n")
        df[col_name] = df[col_name].str.strip("\n")
        df[col_name] = df[col_name].str.strip("\n")
    
    return df



In [490]:
def rest_of_nigeria_job_location(location):
    location_re = re.search(r'\((.*?)\)', location)
    try:
        cleaned_location = location_re.group(0).strip("()")
        return cleaned_location

    except AttributeError as e:
        print(f"No bracket locations with Rest of Nigeria\nLocation value = {location}")
        return location
        
    
#df["Location"] = df["Location"].apply(rest_of_nigeria_job_location)

In [491]:

def convert_dollar_naira(*dollar_vals):
    exchange_rate_api_url = 'https://v6.exchangerate-api.com/v6/f1832750ec5f421a14e74fe8/latest/USD'
    
    max_retries = 3
    current_try = 1
    # Making our request
    while max_retries > 0:
        print(f"Trying exchange rate api. Try number {current_try}")
        try:
            response = requests.get(exchange_rate_api_url)
            break
        except (ConnectionAbortedError, ConnectionResetError, ConnectionError, TimeoutError, OSError) as e:
            print(f"Connection to exchange rate api had an error.\nPython error: {e}")
            
        max_retries -= 1
        current_try += 1
        
        time.sleep(10)
    
    if current_try >= 3:
        naira_rate = 1000
    
    else:
        data = response.json()
        naira_rate = data["conversion_rates"]["NGN"]
    
    naira_vals = []
    for dollar_val in dollar_vals:
        naira_vals.append(float(dollar_val) * naira_rate)

    return naira_vals



def clean_salary(salary):
    
    if salary == 0:
        return "Not given"
    
    if salary == "Not given":
        return salary
    
    salary_1 = salary.strip("\n").strip("+")
    if salary_1.lower() == "commission only":
        return salary_1
    
    salary_2 = (salary_1.partition("than")[2] if "than" in salary_1 else salary_1)
    
    try:
        salary_2_1 = (salary_2, 0 if "than" in salary_1 else salary_1.split("-"))[-1][0] 
        salary_2_2 = (salary_2, 0 if "than" in salary_1 else salary_1.split("-"))[-1][-1]
        
    except TypeError as e:
        print(f"Encountered Type error.\nPython error:{e}\nRemoving the extra slicing term")
        salary_2_1 = (salary_2, 0 if "than" in salary_1 else salary_1.split("-"))[-1] 
        salary_2_2 = (salary_2, 0 if "than" in salary_1 else salary_1.split("-"))[-1]
        
    print(salary_2)
    print(salary_2_1)
    print(salary_2_2)
    
    if salary_2_2 == 0:
        salary_3 = salary_2.strip(" ")

        salary_4 = salary_3.replace(",", "")
        
        if isinstance(salary_4, str):
            return salary_4

        if float(salary_4) < 20_000:
            salary_5 = convert_dollar_naira(salary_4)
            return salary_5[0]
        
        else:
            return salary_4
        
    
    else:
        salary_3 = (salary_2_1.strip(" "), salary_2_2.strip(" "))
        salary_4 = (salary_3[0].replace(",", ""), salary_3[1].replace(",", ""))
        if isinstance(salary_4, str):
            return salary_4
        
        if float(salary_4[1]) < 20_000 and salary_4[0] != salary_4[1]:
            salary_5 = convert_dollar_naira(salary_4[0], salary_4[1])
            salary_range = f"{salary_5[0]} - {salary_5[1]}"
            
            return salary_range
        
        #salary_range = f"{salary_4[0]} - {salary_4[1]}"
        
        return salary_4[0]

In [492]:
def salary_rename(dframe):
    df = dframe.copy()
    df.rename(columns = {"Salary": "Salary_Range"}, inplace=True)

    df["Salary"] = df["Salary_Range"].fillna(0).apply(clean_salary)

    df["Salary"] = df["Salary"].str.split("-").str[0]

    df["Salary_Range"] = df["Salary_Range"].str.strip("\n")
    
    return df

In [493]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    
    messages = [{"role": "user", "content": prompt}]
    retries = 3
    
    while retries > 0:
        try:
            response = openai.ChatCompletion.create(model=model,
                                                    messages=messages,
                                                    temperature=0.9,)
            
        except RateLimitError as e:
            print(f"You have reached the rate limit for today.\nPython error: {e}")
            
            global rate_limit_exceeded
            rate_limit_exceeded = True
            return "Openai error"
        
        except tuple(list_of_openai_errors) as e:
            print(e)
            response = "Openai error"
            time.sleep(20)
            
        if response != "Openai error":
            content_value = response.choices[0].message["content"]
            print(f"Recovered from the error. Generated response: {content_value}")
            return response.choices[0].message["content"]
            
        retries -= 1
        print(f"Num retries = {3 - (retries-1)}/3")
    if isinstance(response, str):
        return response
    else:
        content_value = response.choices[0].message["content"]
        return content_value 




def create_prompt(content):
    
    prompt = f"""{content}\n\nGiven the above description, I would like you to extract, if possible, the experience level (('Entry level', 'No Experience', 'Mid level', 'Senior level',  'Internship & Graduate', 'Executive level'), educational requirements (Degree, HND, High School (S.S.C.E), OND, N.C.E, Diploma, Others, MBA / MSc, MBBS, Vocational), experience length (x years) and salary.

    I want the result to look like this: ["Mid level", "BSc", "4 years", "100,000"]. If any of the values is missing, simply put in an empty string. For example, if the number of years of experience is missing, the result should be this way: ["Mid level", "BSc", "Not given", "100,000"]

    If no information regarding what I asked is given, return ["Not given","Not given","Not given","Not given"]. I don't need any explanation. I only want the result I asked of you in the list format. Also, I need the result to be exactly what I asked. If it says 3 minimum years in the text, I want you to return 3 years. I don't need any extra information. """
    
    return prompt



def jd_information(dframe):
    df = dframe.copy()
    
    mask1 = df["Experience_Length_Requirements"].isnull()
    mask2 = df["Experience_Level_Requirements"].isnull()
    mask3 = df["Education_Requirements"].isnull()
    mask4 = df["Salary"].isnull()
    
    mask = max(mask1.sum(), mask2.sum(), mask3.sum(), mask4.sum())
    
    print(mask)
  
    list_of_jd = list(df.loc[(mask1)| (mask2) | (mask3) | (mask4), :]["Job_Description"].unique())
    
    dict_of_jd = {}
    
    for i, jd in enumerate(list_of_jd):
        dict_of_jd[jd] = []
    
        prompt = create_prompt(jd)
    
        try:
            response = eval(get_completion(prompt))

        except Exception as e:
            dict_of_jd[jd].extend(["Not given","Not given","Not given","Not given"])
            print(e)

        dict_of_jd[jd].extend(response)
        print(f"Done with jd number {i}")
        if i+1 % 2:
            time.sleep(3)
        else:
            time.sleep(62)

    return dict_of_jd, list_of_jd
    


def extract_from_jd(dframe):
    
    df = dframe.copy()
    
    dict_of_jd, list_of_jd = jd_information(df)
    
    df = dframe.copy()
    mask1 = df["Experience_Length_Requirements"].isnull().sum()
    mask2 = df["Experience_Level_Requirements"].isnull().sum()
    mask3 = df["Education_Requirements"].isnull().sum()
    mask4 = df["Salary"].isnull().sum()
    
    if (mask1 + mask2 + mask3 + mask4) > 0:
        for jd in list_of_jd:
            df["Experience_Level_Requirements"] = np.where(df["Job_Description"] == jd, dict_of_jd[jd][0], 
                                                           df["Experience_Level_Requirements"])

            df["Experience_Length_Requirements"] = np.where(df["Job_Description"] == jd, dict_of_jd[jd][2], 
                                                           df["Experience_Length_Requirements"])

            df["Education_Requirements"] = np.where(df["Job_Description"] == jd, dict_of_jd[jd][1], 
                                                           df["Education_Requirements"])

            df["Salary"] = np.where(df["Job_Description"] == jd, dict_of_jd[jd][-1], 
                                                           df["Salary"])
            
        return df
    
    
            
    else:
        return df

In [494]:
def convert_day_posted(day_posted):
    if 'days' in day_posted.lower():
        days = int(day_posted.split()[0])
        return timedelta(days=days)
    elif 'week' in day_posted.lower():
        weeks = int(day_posted.split()[0])
        return timedelta(weeks=weeks)
    elif 'month' in day_posted.lower():
        months = int(day_posted.split()[0])
        return timedelta(days=30 * months)
    
    elif "year" in day_posted.lower():
        years = int(day_posted.split()[0])
        return timedelta(days = 365 * years)
    
    elif "yesterday" in day_posted.lower():
        return timedelta(days = 1)
    
    elif "today" in day_posted.lower():
        return timedelta(days = 0)
    

def resolve_dates(dframe):
    df = dframe.copy()
        
    df['day_posted_timedelta'] = df['Date_Posted'].apply(convert_day_posted)
    
    df['Day_Posted'] = pd.to_datetime(df['Scrape_Time']) - df['day_posted_timedelta']
    
    df["Day_Posted"] = df["Day_Posted"].dt.date
    
    df.drop(["day_posted_timedelta"], axis = 1, inplace = True)
    
    return df

In [495]:
def create_generic_job_prompt(job_name, industry, job_description, lengthy = False):
    
    if not lengthy:
        prompt = f"""I have the following job roles:
        'social media personnel/marketer/digital marketer', 'sales representative', 'personal assistant/secretary/receptionist', 'financial officer', 'graphics designer', 'teacher', 'fashion designer (hair/clothes)', 'logistics officer', 'other', 'lawyer', 'hospitality/culinary personnel', 'doctor/nurse', 'Inventory Manager/Store Keeper', 'laboratory administrator', 'pharmacist', 'business developer', 'inventory manager/store keeper', 'banker', 'it/technical/engineering support', 'Human Resource Officer/Hiring Manager', 'Administrative Officer', 'Intern', 'Housekeeper', 'Architect/Engineer', 'Beautician/Therapist', 'Software Developer/Engineer', 'Vocational Worker', 'Data Expert', 'Scientist', 'IT Security Officer', 'Writer', 'Chief Operating Officer', 'Stocks/FX Trader', 'Consultant', 'Relationship Manager', 'CEO/MD (Chief Executive Officer/Managing Director)', 'Product Management Officer', 'Hospitality/Culinary Personnel', 'Architect/Engineer/Technician'

        I want to fit a new job into 1 of these job role groupings. Here's a description of the new job:
        job name = {job_name}
        industry = {industry}
        job description = {job_description}

        If you cannot ascertain which job role the new job should fit into, return the 'other' job role
        I don't want any explanation. I don't want a phrase or clause. If the result is "banker", I expect you to output "Banker" with no pretext or context."""
        
    else:
        prompt = f"""
        I don't want any extra information just give me the job roles as output alone. Not one word more. Here are the job roles again:
        'social media personnel/marketer/digital marketer', 'sales representative', 'personal assistant/secretary/receptionist', 'financial officer', 'graphics designer', 'teacher', 'fashion designer (hair/clothes)', 'logistics officer', 'other', 'lawyer', 'hospitality/culinary personnel', 'doctor/nurse', 'Inventory Manager/Store Keeper', 'laboratory administrator', 'pharmacist', 'business developer', 'inventory manager/store keeper', 'banker', 'it/technical/engineering support', 'Human Resource Officer/Hiring Manager', 'Administrative Officer', 'Intern', 'Housekeeper', 'Architect/Engineer', 'Beautician/Therapist', 'Software Developer/Engineer', 'Vocational Worker', 'Data Expert', 'Scientist', 'IT Security Officer', 'Writer', 'Chief Operating Officer', 'Stocks/FX Trader', 'Consultant', 'Relationship Manager', 'CEO/MD (Chief Executive Officer/Managing Director)', 'Product Management Officer', 'Hospitality/Culinary Personnel', 'Architect/Engineer/Technician'
        """

    return prompt


def chatgpt_response_check(response):
    try:
        if response.choices and response.choices[0].message and 'content' in response.choices[0].message:
            generated_text = response.choices[0].message['content']
        else:
            generated_text = "No valid response"
            
    except (AttributeError, Exception) as e:
        print(f"The response is already a string.\nPython error:{e}")
        
        generated_text = response

    return generated_text



def create_generic_job_name(df):
    
    job_names = list(df["Job_Name"])
    industries = list(df["Industry"])
    job_descriptions = list(df["Job_Description"])
    
    dict_of_generics = {}
    
    for idx in range(len(job_names)):
        job_name = job_names[idx]
        industry = industries[idx]
        job_description = job_descriptions[idx]
        
        prompt = create_generic_job_prompt(job_name, industry, job_description)
        
        response = get_completion(prompt)
        
        global rate_limit_exceeded 
        
        if rate_limit_exceeded:
            break
        
        retries = 3
        
        while retries > 0:
            if len(response) <= 50:
                break

            prompt = create_generic_job_prompt(job_name, industry, job_description, lengthy=True)
            response = get_completion(prompt)
            retries -= 1
            
        response = chatgpt_response_check(response)
        
        dict_of_generics[job_name] = []
        
        dict_of_generics[job_name].append(response)
        
        print(f"Done with {idx}")
        
        time.sleep(22)
        
        
        
    df["Generic_Job_Name"]= df["Job_Name"].map(dict_of_generics)
    
    
    return df
        

In [496]:
def correct_openai_format(string_data):
    start_index = string_data.find('"content": "') + len('"content": "')
    end_index = string_data.find('"', start_index)
    content = string_data[start_index:end_index]
    
    return content


def columns_check(dframe):
    df = dframe.copy()
    
    try:
        if df["Generic_Job_Name"].str.len().max() > 55:
            df["Generic_Job_Name"] = np.where(df["Generic_Job_Name"].str.contains("<OpenAIObject"), 
                                               df["Generic_Job_Name"].apply(correct_openai_format), df["Generic_Job_Name"])

        if df["Generic_Job_Name"].str.startswith("[").sum() > 0:
            print("remvoing brackets")
            df["Generic_Job_Name"] = df["Generic_Job_Name"].str.replace("[", "")
            df["Generic_Job_Name"] = df["Generic_Job_Name"].str.replace("]", "")
            
        if df["Generic_Job_Name"].str.startswith("'").sum() > 0:
            #df["Generic_Job_Name"] = df["Generic_Job_Name"].str.strip()
            df["Generic_Job_Name"] = df["Generic_Job_Name"].str.strip("''")

            
        if df["Scrape_Time"].dtype == "O":
            try:
                df["Scrape_Time"] = pd.to_datetime(df["Scrape_Time"], format='mixed', dayfirst= True)
                
            except ValueError as e:
                print(f"Encountered an error.\nPython error: {e}")
                df = checking_recorded_column_names(df)
                
            

        if df["Day_Posted"].dtype == "O":
            df["Day_Posted"] = pd.to_datetime(df["Day_Posted"])
            
    except Exception as e:
        warnings.warn("column check could not be completed")
        print(f"Python error: {e}")
              
    
    return df
        

        

In [497]:
# try: 
#     df["Scrape_Time"] = pd.to_datetime(df["Scrape_Time"], format='mixed', dayfirst= True)
    
# except ValueError as e:
#     print(e)

In [498]:
# response = "abc"

# if response.choices and response.choices[0].message and 'content' in response.choices[0].message:
#     generated_text = response.choices[0].message['content']
# else:
#     generated_text = "No valid response"

In [499]:
def remove_raw_data_from_path(file_path):
    
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"File '{file_path}' has been deleted.")
    
    else:
        print(f"File '{file_path}' does not exist.")


In [500]:
# def transfer_file_to_container(filename, blob_name, container_name, connection_string):
#     # Import the client object from the SDK library
#     from azure.storage.blob import BlobClient

#     # Retrieve the connection string from an environment variable. Note that a
#     # connection string grants all permissions to the caller, making it less
#     # secure than obtaining a BlobClient object using credentials.
#     connection_string = connection_string

#     # Create the client object for the resource identified by the connection
#     # string, indicating also the blob container and the name of the specific
#     # blob we want.
#     blob_client = BlobClient.from_connection_string(
#         connection_string,
#         container_name= container_name,
#         blob_name= blob_name,
#     )

#     # Open a local file and upload its contents to Blob Storage
#     with open(f"./{filename}", "rb") as data:
#         blob_client.upload_blob(data)
#         print(f"Uploaded {filename} to {blob_client.url}")


        
# def move_file_to_container(filename):
#     current_directory = os.getcwd()
#     files_in_directory = os.listdir(current_directory)
    
#     if filename in files_in_directory:
#         print(f"{filename} exists in the current directory")
#         transfer_file_to_container(filename, filename, container_name, connection_string)
        
#     else:
#         print(f"{filename} is not in the current directory")

In [501]:
# if isinstance(df_, str):
#     print("No file in the path to clean")

# else:
#     if len(df_) > 20:
#         begin = 0
#         for i in range(begin, len(df_), 20):
#             if i == 0:
#                 continue
#             else:
#                 small_data = df_.iloc[begin:i, :]
#                 small_data.to_csv(f"Jobberman_data_raw{i}.csv", index = False)
#                 begin = i
#                 print(small_data)
#             print("Cleaning and Transforming in chunks of 20")
            
                


In [502]:
if isinstance(df, str):
    print("No file in the path to clean")

else:
    chunk_size = 20  # Chunk size for splitting DataFrame
    if len(df) > chunk_size:
        for i in range(0, len(df), chunk_size):
            small_data = df.iloc[i:i + chunk_size, :]
            print(i)
            #print(small_data)
            small_data.to_csv(f"Jobberman_data_raw{i}.csv", index = False)
            if small_data.empty:
                break
            
            print("Cleaning and Transforming in chunks of 20")
            if i <= 160:
                small_df = pd.read_csv(f"Jobberman_data_raw{i}.csv")
                small_df = checking_recorded_column_names(small_df)#step1
                print("Finished step 1: Checking recorded column names")

                small_df = remove_new_line(small_df)#step 2
                print("Finished step 2: Removing newline characters")

                small_df["Location"] = small_df["Location"].apply(rest_of_nigeria_job_location)#step 3
                print("Finished step 3: Cleaning up the Location column")

                small_df = salary_rename(small_df)#step 4
                print("Finished step 4: Creating and cleaning Salary and Salary range columns respectively")

                small_df = extract_from_jd(small_df)#step 5
                print("Finished step 5: Using Chatgpt to extract missing details from job description")

                small_df = resolve_dates(small_df)#step 6
                print("Finished step 6: Done resolving the date issues using scrape time")

                small_df = create_generic_job_name(small_df)#step 7
                print("Finished step 7: Assigning generic job names to each posting")

                small_df.fillna("Not given", inplace = True)#step 8
                print("Finished step 8: Filling up all other missing values with 'Not given'")
                
                print("Checking some columns for format issues...")
                small_df["Generic_Job_Name"] = small_df["Generic_Job_Name"].apply(lambda x: x[0] if len(x)==1 else x)
                small_df = columns_check(small_df)
                remainder1_df = small_df[small_df["Generic_Job_Name"].isnull()]
                small_df = small_df.loc[~small_df["Generic_Job_Name"].isnull(), :]

                small_df = small_df[['Job_Name', 'Generic_Job_Name', 'Company_Name', 'Industry', 'Location', 'Job_Type',
                   'Job_Summary', 'Education_Requirements',
                   'Experience_Level_Requirements', 'Experience_Length_Requirements',
                   'Job_Description', 'Salary', 'Salary_Range', 'Date_Posted', 'Day_Posted', 'Scrape_Time', 'url',
                   ]]#step 9
                print("Finished step 9: Reordering the column names to match the database")


#                 df.to_csv("./Cleaned_Jobberman.csv", index = False)#step 10
#                 print("Finished step 10: Creating a cleaned CSV version of the scraped data")

                small_df.to_csv("./Completed_Cleaned_Jobberman.csv", index= False, mode = "a")
                print("Updated the completed cleaned jobberman csv file")
            
            else: 
                break
#                 df.to_parquet("Cleaned_Jobberman.parquet")
#                 print("Created the cleaned parquet file for backup")

        

    
    else:
        df = checking_recorded_column_names(df)#step1
        print("Finished step 1: Checking recorded column names")

        df = remove_new_line(df)#step 2
        print("Finished step 2: Removing newline characters")

        df["Location"] = df["Location"].apply(rest_of_nigeria_job_location)#step 3
        print("Finished step 3: Cleaning up the Location column")

        df = salary_rename(df)#step 4
        print("Finished step 4: Creating and cleaning Salary and Salary range columns respectively")

        df = extract_from_jd(df)#step 5
        print("Finished step 5: Using Chatgpt to extract missing details from job description")

        df = resolve_dates(df)#step 6
        print("Finished step 6: Done resolving the date issues using scrape time")

        df = create_generic_job_name(df)#step 7
        remainder1_df = df[df["Generic_Job_Name"].isnull()]
        df = df.loc[~small_df["Generic_Job_Name"].isnull(), :]

        print("Finished step 7: Assigning generic job names to each posting")

        df.fillna("Not given", inplace = True)#step 8
        print("Finished step 8: Filling up all other missing values with 'Not given'")
        
        print("Checking some columns for format issues...")
        df["Generic_Job_Name"] = df["Generic_Job_Name"].apply(lambda x: x[0] if len(x)==1 else x)
        df = columns_check(df)

        df = df[['Job_Name', 'Generic_Job_Name', 'Company_Name', 'Industry', 'Location', 'Job_Type',
           'Job_Summary', 'Education_Requirements',
           'Experience_Level_Requirements', 'Experience_Length_Requirements',
           'Job_Description', 'Salary', 'Salary_Range', 'Date_Posted', 'Day_Posted', 'Scrape_Time', 'url',
           ]]#step 9
        print("Finished step 9: Reordering the column names to match the database")


#         df.to_csv("./Cleaned_Jobberman.csv", index = False)#step 10
#         print("Finished step 10: Creating a cleaned CSV version of the scraped data")
        
        df.to_csv("./Completed_Cleaned_Jobberman.csv", index= False, mode = "a")#step 11
        print("Updated the completed cleaned jobberman csv file")

#         df.to_parquet("Cleaned_Jobberman.parquet")#step 11
#         print("Created the cleaned parquet file for backup")



    try:
        completed_df = pd.read_csv("Completed_Cleaned_Jobberman.csv")
        if 'Jobberman_data_raw.csv' in os.listdir():
            df = pd.read_csv("Jobberman_data_raw.csv")
            
        elif "Jobberman_data_raw_remainder.csv" in os.listdir():
            df = pd.read_csv("Jobberman_data_raw_remainder.csv")
            
        if len(set(df["url"])) == len(set(completed_df["url"])):
            print("Cleaning and transformation completed")

        elif len(set(df["url"])) > len(set(completed_df["url"])):
            remainder_df = df.loc[~df["url"].isin(list(completed_df["url"])), :]
            
            if remainder1_df.empty:
                remainder_df = remainder_df.drop_duplicates()
                remainder_df.to_csv("Jobberman_data_raw_remainder.csv", index = False)
            else:
                remainder1_df = remainder1_df[remainder_df.columns]
                remainder_df = pd.concat([remainder_df, remainder1_df])
                remainder_df = remainder_df.drop_duplicates()
                remainder_df.to_csv("Jobberman_data_raw_remainder.csv", index = False)

        else:
            print("Already processed. File not cleaned up!")
            remainder_df = completed_df.loc[~completed_df["url"].isin(list(df["url"])), :]
            
            if remainder1_df.empty:
                remainder_df = remainder_df.drop_duplicates()
                remainder_df.to_csv("Jobberman_data_raw_remainder.csv", index = False)
                
            else:
                remainder1_df = remainder1_df[remainder_df.columns]
                remainder_df = pd.concat([remainder_df, remainder1_df])
                remainder_df = remainder_df.drop_duplicates()
                remainder_df.to_csv("Jobberman_data_raw_remainder.csv", index = False)


    except FileNotFoundError as e:
        print(f"Completed Cleaned Jobberman csv file could not be found.\nPython error: {e}")
        #print("Removing the raw CSV file now...")
        #remove_raw_data_from_path("./Jobberman_data_raw.csv")#step 11

    
    try:
        completed_df.to_parquet("Completed_Cleaned_Jobberman.parquet", index= False)
        
    except FileNotFoundError as e:
        print(f"Completed Cleaned Jobberman csv file could not be found.\nPython error: {e}")
    



        #df.to_parquet("./Cleaned_Jobberman.parquet")



        #move_file_to_container("./Cleaned_Jobberman.parquet")

0
Cleaning and Transforming in chunks of 20
Finished step 1: Checking recorded column names
Finished step 2: Removing newline characters
No bracket locations with Rest of Nigeria
Location value = Abuja
No bracket locations with Rest of Nigeria
Location value = Lagos
No bracket locations with Rest of Nigeria
Location value = Abuja
No bracket locations with Rest of Nigeria
Location value = Lagos
No bracket locations with Rest of Nigeria
Location value = Lagos
No bracket locations with Rest of Nigeria
Location value = Lagos
No bracket locations with Rest of Nigeria
Location value = Lagos
No bracket locations with Rest of Nigeria
Location value = Lagos
No bracket locations with Rest of Nigeria
Location value = Abuja
No bracket locations with Rest of Nigeria
Location value = Abuja
No bracket locations with Rest of Nigeria
Location value = Abuja
No bracket locations with Rest of Nigeria
Location value = Lagos
No bracket locations with Rest of Nigeria
Location value = Abuja
No bracket locatio

Recovered from the error. Generated response: IT/Technical/Engineering Support
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 14
Recovered from the error. Generated response: Administrative Officer
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 15
Recovered from the error. Generated response: Logistics Officer
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 16
Recovered from the error. Generated response: Driver
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 17
Recovered from the error. Generated response: Digital Marketer and Social Media Administrator
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 18
Recovered from the error. Generated response: IT Security Officer
The response is already a string.
Python error:'str' object has no attribut

Recovered from the error. Generated response: Technician, Network Physical Infrastructures fits into the job role grouping of 'it/technical/engineering support'.
Recovered from the error. Generated response: 'social media personnel/marketer/digital marketer', 'sales representative', 'personal assistant/secretary/receptionist', 'financial officer', 'graphics designer', 'teacher', 'fashion designer (hair/clothes)', 'logistics officer', 'other', 'lawyer', 'hospitality/culinary personnel', 'doctor/nurse', 'Inventory Manager/Store Keeper', 'laboratory administrator', 'pharmacist', 'business developer', 'inventory manager/store keeper', 'banker', 'it/technical/engineering support', 'Human Resource Officer/Hiring Manager', 'Administrative Officer', 'Intern', 'Housekeeper', 'Architect/Engineer', 'Beautician/Therapist', 'Software Developer/Engineer', 'Vocational Worker', 'Data Expert', 'Scientist', 'IT Security Officer', 'Writer', 'Chief Operating Officer', 'Stocks/FX Trader', 'Consultant', 'Re

Recovered from the error. Generated response: IT/Technical/Engineering Support
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 15
Recovered from the error. Generated response: Architect/Engineer/Technician
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 16
Recovered from the error. Generated response: Technician, Network Physical Infrastructures
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 17
Recovered from the error. Generated response: IT Support Officer
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 18
Recovered from the error. Generated response: Other
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 19
Finished step 7: Assigning generic job names to each posting
Finished step 8: Filling up all other missing values with 'Not given'
Checki

Recovered from the error. Generated response: IT Support Technician
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 0
Recovered from the error. Generated response: Other
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 1
Recovered from the error. Generated response: Financial Officer
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 2
Recovered from the error. Generated response: Teacher
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 3
Recovered from the error. Generated response: Social media personnel/marketer/digital marketer
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 4
Recovered from the error. Generated response: Accountant
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 5
Recovered from t

Recovered from the error. Generated response: Software Developer/Engineer
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 13
Recovered from the error. Generated response: Administrative Officer
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 14
Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
Num retries = 2/3
Recovered from the error. Generated response: Chef
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 15
Recovered from the error. Generated response: Financial Controller
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 16
Recovered from the error. Generated response: Other
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 17
Recovered from the error. Generated response: Financial Offic

Recovered from the error. Generated response: Other
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 0
Recovered from the error. Generated response: Sales Representative
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 1
Recovered from the error. Generated response: Administrative Officer
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 2
Recovered from the error. Generated response: Digital Marketing Intern
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 3
Recovered from the error. Generated response: Other
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 4
Recovered from the error. Generated response: Other
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 5
Recovered from the error. Generated respons

Recovered from the error. Generated response: Administrative Officer
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 15
Recovered from the error. Generated response: Logistics Officer
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 16
Recovered from the error. Generated response: Laboratory Administrator
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 17
Recovered from the error. Generated response: Software Developer/Engineer
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 18
Recovered from the error. Generated response: Scrum Master
The response is already a string.
Python error:'str' object has no attribute 'choices'
Done with 19
Finished step 7: Assigning generic job names to each posting
Finished step 8: Filling up all other missing values with 'Not given'
Checking some columns for format

In [503]:
df

,Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Date_Posted,Scrape_Time,url
0,Driver,\n\nICST Solutions\n\n,\n\nEngineering & Technology\n\n,Abuja,Full Time,We seek to hire a suitable candidate to fill t...,High School (S.S.C.E),Mid level,4 years,Diploma/ SSCE or O’levels equivalentA minimum ...,"\nLess than 75,000\n",3 days ago,2023-11-13 05:51:49,https://www.jobberman.com/listings/driver-840j2p
1,Driver,\n\nICST Solutions\n\n,\n\nEngineering & Technology\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,High School (S.S.C.E),Mid level,4 years,Diploma/ SSCE or O’levels equivalentA minimum ...,"\nLess than 75,000\n",3 days ago,2023-11-13 05:51:59,https://www.jobberman.com/listings/driver-q2qmkz
2,Digital Marketer And Social Media Administrator,\n\nICST Solutions\n\n,\n\nMarketing & Communications\n\n,Abuja,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,3 years,Develop marketing strategies that align with t...,"\n150,000 - 250,000\n",3 days ago,2023-11-13 05:52:10,https://www.jobberman.com/listings/digital-mar...
3,Chief Information Security Officer,\n\nJubilee-Life Mortgage Bank\n\n,\n\nEngineering & Technology\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,8 years,"Responsible for\nidentifying, developing, impl...","\n400,000 - 600,000\n",3 days ago,2023-11-13 05:52:20,https://www.jobberman.com/listings/chief-infor...
4,Sales Officer (Network Foundation and Voice an...,\n\nICST Solutions\n\n,\n\nSales\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Senior level,4 years,To reach and exceed all quarterly and annual t...,"\n250,000 - 400,000\n",3 days ago,2023-11-13 05:52:31,https://www.jobberman.com/listings/sales-offic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Senior Software Engineer,\n\nAyulla Inc.\n\n,\n\nSoftware & Data\n\n,Remote (Work From Home),Full Time,We are looking for a talented Python/Django ba...,Degree,Senior level,5 years,"Develop well-designed, scalable, and robust so...","\n400,000 - 600,000\n",1 week ago,2023-11-13 06:01:42,https://www.jobberman.com/listings/senior-soft...
246,Marketing Executive,\n\nLifemate Nigeria Limited\n\n,\n\nMarketing & Communications\n\n,Abeokuta & Ogun State,Full Time,We are looking to hire a suitable candidate to...,OND,Entry level,1 year,Identify and meet with furniture dealers to es...,"\n75,000 - 150,000\n",1 week ago,2023-11-13 06:01:51,https://www.jobberman.com/listings/marketing-e...
247,Architect,\n\nLifemate Nigeria Limited\n\n,\n\nBuilding & Architecture\n\n,Lagos,Full Time,We are looking to hire a suitable candidate to...,Degree,Entry level,1 year,Manage the company’s corporate prototype desig...,"\n75,000 - 150,000\n",1 week ago,2023-11-13 06:02:00,https://www.jobberman.com/listings/architect-x...
248,React Native Developer,\n\nLeaders Network\n\n,\n\nSoftware & Data\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,2 years,Mobile App Development: Design and develop mob...,"\n150,000 - 250,000\n",1 week ago,2023-11-13 06:02:09,https://www.jobberman.com/listings/react-nativ...


In [504]:
# completed_df["Generic_Job_Name"].str.len().max()

completed_df

,Job_Name,Generic_Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Salary_Range,Date_Posted,Day_Posted,Scrape_Time,url
0,Driver,Driver,ICST Solutions,Engineering & Technology,Abuja,Full Time,We seek to hire a suitable candidate to fill t...,High School (S.S.C.E),Mid level,4 years,Diploma/ SSCE or O’levels equivalentA minimum ...,75000,"Less than 75,000",3 days ago,2023-11-10,2023-11-13 05:51:49,https://www.jobberman.com/listings/driver-840j2p
1,Driver,Driver,ICST Solutions,Engineering & Technology,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,High School (S.S.C.E),Mid level,4 years,Diploma/ SSCE or O’levels equivalentA minimum ...,75000,"Less than 75,000",3 days ago,2023-11-10,2023-11-13 05:51:59,https://www.jobberman.com/listings/driver-q2qmkz
2,Digital Marketer And Social Media Administrator,Digital Marketer and Social Media Administrator,ICST Solutions,Marketing & Communications,Abuja,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,3 years,Develop marketing strategies that align with t...,150000,"150,000 - 250,000",3 days ago,2023-11-10,2023-11-13 05:52:10,https://www.jobberman.com/listings/digital-mar...
3,Chief Information Security Officer,IT Security Officer,Jubilee-Life Mortgage Bank,Engineering & Technology,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,8 years,"Responsible for\nidentifying, developing, impl...",400000,"400,000 - 600,000",3 days ago,2023-11-10,2023-11-13 05:52:20,https://www.jobberman.com/listings/chief-infor...
4,Sales Officer (Network Foundation and Voice an...,Sales Representative,ICST Solutions,Sales,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Senior level,4 years,To reach and exceed all quarterly and annual t...,250000,"250,000 - 400,000",3 days ago,2023-11-10,2023-11-13 05:52:31,https://www.jobberman.com/listings/sales-offic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Client Liaison Officer,Administrative Officer,Nordica Fertility Centre,Human Resources,Lagos,Full Time,We are looking to hire a suitable candidate to...,HND,Mid level,3 years,Enquiry TakingEnquiry FLUFLU on way forwardIVF...,150000,"150,000 - 250,000",1 week ago,2023-11-06,2023-11-13 06:00:41,https://www.jobberman.com/listings/client-liai...
184,Import/Export Coordinator,Logistics Officer,SUNAGROW International Oil,Supply Chain & Procurement,Kaduna,Full Time,He/She is responsible for the coordination and...,HND,Mid level,5 years,"Maintains shipping records, keeps accurate doc...",150000,"150,000 - 250,000",1 week ago,2023-11-06,2023-11-13 06:00:50,https://www.jobberman.com/listings/importexpor...
185,Embryologist,Laboratory Administrator,Nordica Fertility Centre,Medical & Pharmaceutical,Lagos,Full Time,We are looking to hire a suitable candidate to...,Degree,Mid level,3 years,Daily cleaning and maintenance of the embryolo...,150000,"150,000 - 250,000",1 week ago,2023-11-06,2023-11-13 06:00:58,https://www.jobberman.com/listings/embryologis...
186,Node JS Developer,Software Developer/Engineer,Leaders Network,Software & Data,Lagos,Full Time,A Node.js Developer is responsible for designi...,Degree,Entry level,1 year,Node.js Application Development: Design and de...,150000,"150,000 - 250,000",1 week ago,2023-11-06,2023-11-13 06:01:08,https://www.jobberman.com/listings/node-js-dev...


In [505]:
# small_df["Generic_Job_Name"].str.len().max()
remainder_df

,Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Date_Posted,Scrape_Time,url
186,Frontend Developer,\n\nLeaders Network\n\n,\n\nSoftware & Data\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Entry level,2 years,Develop User Interfaces: Create user-friendly ...,"\n150,000 - 250,000\n",1 week ago,2023-11-13 05:57:21,https://www.jobberman.com/listings/frontend-de...
187,Direct Sales Personnel,\n\nKirby Nigeria\n\n,\n\nSales\n\n,Abuja,Full Time,We seek to hire a suitable candidate to fill t...,Others,Entry level,2 years,BSc/HND in Business or related fieldMinimum of...,"\n75,000 - 150,000\n",1 week ago,2023-11-13 05:57:30,https://www.jobberman.com/listings/direct-sale...
188,PHP Developer,\n\nLeaders Network\n\n,\n\nSoftware & Data\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,2 years,Web Application Development: Design and develo...,"\n150,000 - 250,000\n",1 week ago,2023-11-13 05:57:40,https://www.jobberman.com/listings/php-develop...
189,Graphic Designer (Intern),\n\nMicroWeb Tech\n\n,\n\nCreative & Design\n\n,Abuja,Internship & Graduate,This is a full-time on-site internship role fo...,HND,Internship & Graduate,No Experience/Less than 1 year,Minimum academic qualification of HND in any r...,"\nLess than 75,000\n",5 days ago,2023-11-13 05:57:49,https://www.jobberman.com/listings/graphic-des...
190,Social Media Manager Intern,\n\nMicroWeb Tech\n\n,\n\nCreative & Design\n\n,Abuja,Internship & Graduate,This is a full-time on-site internship role fo...,HND,Internship & Graduate,No Experience/Less than 1 year,Minimum academic qualification of HND in any r...,"\nLess than 75,000\n",5 days ago,2023-11-13 05:57:58,https://www.jobberman.com/listings/social-medi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Senior Software Engineer,\n\nAyulla Inc.\n\n,\n\nSoftware & Data\n\n,Remote (Work From Home),Full Time,We are looking for a talented Python/Django ba...,Degree,Senior level,5 years,"Develop well-designed, scalable, and robust so...","\n400,000 - 600,000\n",1 week ago,2023-11-13 06:01:42,https://www.jobberman.com/listings/senior-soft...
246,Marketing Executive,\n\nLifemate Nigeria Limited\n\n,\n\nMarketing & Communications\n\n,Abeokuta & Ogun State,Full Time,We are looking to hire a suitable candidate to...,OND,Entry level,1 year,Identify and meet with furniture dealers to es...,"\n75,000 - 150,000\n",1 week ago,2023-11-13 06:01:51,https://www.jobberman.com/listings/marketing-e...
247,Architect,\n\nLifemate Nigeria Limited\n\n,\n\nBuilding & Architecture\n\n,Lagos,Full Time,We are looking to hire a suitable candidate to...,Degree,Entry level,1 year,Manage the company’s corporate prototype desig...,"\n75,000 - 150,000\n",1 week ago,2023-11-13 06:02:00,https://www.jobberman.com/listings/architect-x...
248,React Native Developer,\n\nLeaders Network\n\n,\n\nSoftware & Data\n\n,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,2 years,Mobile App Development: Design and develop mob...,"\n150,000 - 250,000\n",1 week ago,2023-11-13 06:02:09,https://www.jobberman.com/listings/react-nativ...


In [433]:
#small_df

,Job_Name,Generic_Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Salary_Range,Date_Posted,Day_Posted,Scrape_Time,url
0,Accountant,Financial Officer,Hairitage Hub,"Accounting, Auditing & Finance",Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,3 years,Raise requisition of all expenses to be made a...,Not given,Not given,Today,2023-11-06,2023-11-06 19:34:32,https://www.jobberman.com/listings/accountant-...
1,Business Development Executive,Business Developer,Jobberman (Third Party Recruitment),Management & Business Development,Lagos,Full Time,Our client seeks to hire a suitable candidate ...,Degree,Mid level,4 years,Developing and sustaining solid relationships ...,250000,"250,000 - 400,000",Today,2023-11-06,2023-11-06 19:34:43,https://www.jobberman.com/listings/business-de...
2,Accountant,Financial Officer,Wale Adenuga Productions,"Accounting, Auditing & Finance",Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,2 years,"HND/BSc in Accounting, Finance, or relevant de...",150000,"150,000 - 250,000",Today,2023-11-06,2023-11-06 19:34:51,https://www.jobberman.com/listings/accountant-...
3,Social Media Manager,Social Media Personnel/Marketer/Digital Marketer,Wale Adenuga Productions,Marketing & Communications,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,HND,Mid level,2 years,Collaborating with the social media team to de...,75000,"75,000 - 150,000",Today,2023-11-06,2023-11-06 19:35:02,https://www.jobberman.com/listings/social-medi...
4,Marketing Executive,Sales Representative,Wale Adenuga Productions,Marketing & Communications,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,HND,Mid level,2 years,"HND/BSc in Marketing, Communications, Business...",150000,"150,000 - 250,000",Today,2023-11-06,2023-11-06 19:35:12,https://www.jobberman.com/listings/marketing-e...
5,Global Financial Services Sales Representative,Sales Representative,Jobberman (Third Party Recruitment),Sales,Work From Home,Full Time,As a Global Financial Services Sales Represent...,Degree,Mid level,5 years,"Generate leads through various channels, inclu...",400000,"400,000 - 600,000",Today,2023-11-06,2023-11-06 19:35:22,https://www.jobberman.com/listings/global-fina...
6,Admin and Legal Officer,Administrative Officer,Izy Group of Companies Limited,Legal Services,Abuja,Full Time,The Admin and Legal Officer at Izy Air Limited...,Degree,Mid level,3 years,Conduct thorough analysis of evolving industry...,150000,"150,000 - 250,000",Today,2023-11-06,2023-11-06 19:35:31,https://www.jobberman.com/listings/admin-and-l...
7,Digital Marketing Intern,Social Media Personnel/Marketer/Digital Marketer,Olisa Agbakoba Legal (OAL),Marketing & Communications,Lagos,Internship & Graduate,Are you an NYSC member with a passion for shap...,Degree,Internship & Graduate,No Experience/Less than 1 year,Support in planning and execute digital market...,75000,"Less than 75,000",Today,2023-11-06,2023-11-06 19:35:41,https://www.jobberman.com/listings/digital-mar...
8,Plumber,Other,AQUATRIO NIGERIA LTD,Trades & Services,Lagos,Full Time,"This is aquatrio Nigeria Limited, we are the s...",High School (S.S.C.E),Entry level,3 years,Fixing water filter in the company for any of ...,75000,"75,000 - 150,000",Today,2023-11-06,2023-11-06 19:35:56,https://www.jobberman.com/listings/plumber-q2qjwp
9,Marketing Executive,Sales Representative,Hathor Coast Limited,Marketing & Communications,Work From Home,Full Time,We are seeking a full-time Marketing Executive...,HND,Entry level,1 year,"HND/BSc in Marketing, Communications, Business...",75000,"Less than 75,000",Today,2023-11-06,2023-11-06 19:36:41,https://www.jobberman.com/listings/marketing-e...


In [408]:
# x = "Not given"

# if x == 0:
#     print("No error")

In [407]:
#df[df["Job_Name"]== "Cybersecurity Research Engineer"]

,Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Date_Posted,Scrape_Time,url
8,Cybersecurity Research Engineer,NetSPI,Engineering & Technology,Work From Home,Full Time,We are seeking a cybersecurity research engine...,Degree,Mid level,2 years,Work across engineering and operations to iden...,Not given,2 days ago,2023-11-03 08:22:19,https://www.jobberman.com/listings/cybersecuri...
26,Cybersecurity Research Engineer,NetSPI,Engineering & Technology,Work From Home,Full Time,We are seeking a cybersecurity research engine...,Degree,Mid level,2 years,Work across engineering and operations to iden...,Not given,2 days ago,2023-11-03 08:26:20,https://www.jobberman.com/listings/cybersecuri...


In [363]:
# # small_df["Generic_Job_Name"] = small_df["Generic_Job_Name"].apply(lambda x: x[0])

# try:
#     completed_df = pd.read_csv("Completed_Cleaned_Jobberman.csv")
#     if 'Jobberman_data_raw.csv' in os.listdir():
#         df = pd.read_csv("Jobberman_data_raw.csv")

#     elif "Jobberman_data_raw_remainder.csv" in os.listdir():
#         df = pd.read_csv("Jobberman_data_raw_remainder.csv")

#     if len(set(df["url"])) == len(set(completed_df["url"])):
#         print("Cleaning and transformation completed")

#     elif len(set(df["url"])) > len(set(completed_df["url"])):
#         remainder_df = df.loc[~df["url"].isin(list(completed_df["url"])), :]

#         if remainder1_df.empty:
#             remainder_df = remainder_df.drop_duplicates()
#             remainder_df.to_csv("Jobberman_data_raw_remainder.csv", index = False)
#         else:
#             remainder1_df = remainder1_df[remainder_df.columns]
#             remainder_df = pd.concat([remainder_df, remainder1_df])
#             remainder_df = remainder_df.drop_duplicates()
#             remainder_df.to_csv("Jobberman_data_raw_remainder.csv", index = False)

#     else:
#         print("Already processed. File not cleaned up!")
#         remainder_df = completed_df.loc[~completed_df["url"].isin(list(df["url"])), :]

#         if remainder1_df.empty:
#             remainder_df = remainder_df.drop_duplicates()
#             remainder_df.to_csv("Jobberman_data_raw_remainder.csv", index = False)

#         else:
#             remainder1_df = remainder1_df[remainder_df.columns]
#             remainder_df = pd.concat([remainder_df, remainder1_df])
#             remainder_df = remainder_df.drop_duplicates()
#             remainder_df.to_csv("Jobberman_data_raw_remainder.csv", index = False)


# except FileNotFoundError as e:
#     print(f"Completed Cleaned Jobberman csv file could not be found.\nPython error: {e}")
#     #print("Removing the raw CSV file now...")
#     #remove_raw_data_from_path("./Jobberman_data_raw.csv")#step 11


# try:
#     completed_df.to_parquet("Completed_Cleaned_Jobberman.parquet", index= False)

# except FileNotFoundError as e:
#     print(f"Completed Cleaned Jobberman csv file could not be found.\nPython error: {e}")






Already processed. File not cleaned up!


In [364]:
#remainder_df

,Job_Name,Generic_Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Salary_Range,Date_Posted,Day_Posted,Scrape_Time,url
0,Fleet Officer,Other,Brows by Previv Beauty,Management & Business Development,Work From Home,Full Time,TAA Logistics is a last-mile delivery company ...,High School (S.S.C.E),Entry level,1 year,Fleet management: Ensure the optimal utilizati...,75000,"Less than 75,000",Yesterday,2023-11-02,2023-11-03 08:21:06,https://www.jobberman.com/listings/fleet-and-o...
1,Sales Officer,Sales Representative,kejijoe myriad concept,Sales,Lagos,Full Time,We are looking for a results-driven sales repr...,OND,Entry level,2 years,"Present, promote and sell products/services us...",75000,"75,000 - 150,000",Yesterday,2023-11-02,2023-11-03 08:21:14,https://www.jobberman.com/listings/sales-offic...
2,Senior Medical Officer (Internal Medicine),Medical Officer,Dr hassan Hospital & Diagostic Centre,Medical & Pharmaceutical,Abuja,Full Time,As a Senior Medical Officer at Dr. Hassan Hosp...,MBBS,Senior level,5 years,"Lead and supervise a team of medical officers,...",400000,"400,000 - 600,000",Yesterday,2023-11-02,2023-11-03 08:21:22,https://www.jobberman.com/listings/senior-medi...
3,Content Writer,Writer,Young & Cerebral,Creative & Design,Lagos,Full Time,We are looking for a smart and intelligent Con...,HND,Entry level,2 years,"Write clear and concise copy for ads, scripts...",75000,"75,000 - 150,000",Yesterday,2023-11-02,2023-11-03 08:21:30,https://www.jobberman.com/listings/content-wri...
4,Procurement/Admin Personnel,Administrative Officer,StreSERT Integrated Limited,Admin & Office,Lagos,Full Time,The Procurement/ Admin personnel will be respo...,HND,Entry level,2 years,"Procurement: (office supplies, assets, station...",75000,"75,000 - 150,000",Yesterday,2023-11-02,2023-11-03 08:21:39,https://www.jobberman.com/listings/procurement...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Dental Nurse,Not given,Celeb Beauty Clinic,Medical & Pharmaceutical,Lagos,Full Time,We are looking to hire a suitable candidate to...,Degree,Entry level,1 year,"Prepare the dental surgery for each patient, e...",75000,"75,000 - 150,000",Yesterday,2023-11-02,2023-11-03 08:34:07,https://www.jobberman.com/listings/nurse-7wv2xm
184,Social Media Intern (NYSC),Not given,Young & Cerebral,Marketing & Communications,Lagos,Internship & Graduate,We are looking for an enthusiastic and smart i...,HND,No Experience,No Experience/Less than 1 year,Manage and maintain the organization's social ...,75000,"Less than 75,000",Yesterday,2023-11-02,2023-11-03 08:34:17,https://www.jobberman.com/listings/social-medi...
185,IT Officer,IT Officer,Jobberman (Third Party Recruitment),Engineering & Technology,Lagos,Full Time,We are looking for a qualified IT officer that...,Degree,Entry level,2 years,"Installing and configuring computer hardware, ...",Not given,Not given,Yesterday,2023-11-02,2023-11-03 08:34:27,https://www.jobberman.com/listings/it-officer-...
186,Accountant,Not given,Femstone Engineering Ltd,"Accounting, Auditing & Finance",Abuja,Full Time,We are seeking a skilled and experienced accou...,Degree,Entry level,1 year,"Perform day-to-day accounting activities, incl...",75000,"75,000 - 150,000",1 week ago,2023-10-27,2023-11-03 08:28:19,https://www.jobberman.com/listings/accountant-...


In [362]:
#len(set(df["url"])) > len(set(completed_df["url"]))

True

In [360]:
#len(set(df["url"]))

146

In [361]:
#len(set(completed_df["url"]))

89

In [365]:
#completed_df.to_parquet("Completed_Cleaned_Jobberman.parquet", index= False)

In [366]:
#completed_df

,Job_Name,Generic_Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Salary_Range,Date_Posted,Day_Posted,Scrape_Time,url
0,Fleet Officer,Other,Brows by Previv Beauty,Management & Business Development,Work From Home,Full Time,TAA Logistics is a last-mile delivery company ...,High School (S.S.C.E),Entry level,1 year,Fleet management: Ensure the optimal utilizati...,75000,"Less than 75,000",Yesterday,2023-11-02,2023-11-03 08:21:06,https://www.jobberman.com/listings/fleet-and-o...
1,Sales Officer,Sales Representative,kejijoe myriad concept,Sales,Lagos,Full Time,We are looking for a results-driven sales repr...,OND,Entry level,2 years,"Present, promote and sell products/services us...",75000,"75,000 - 150,000",Yesterday,2023-11-02,2023-11-03 08:21:14,https://www.jobberman.com/listings/sales-offic...
2,Senior Medical Officer (Internal Medicine),Medical Officer,Dr hassan Hospital & Diagostic Centre,Medical & Pharmaceutical,Abuja,Full Time,As a Senior Medical Officer at Dr. Hassan Hosp...,MBBS,Senior level,5 years,"Lead and supervise a team of medical officers,...",400000,"400,000 - 600,000",Yesterday,2023-11-02,2023-11-03 08:21:22,https://www.jobberman.com/listings/senior-medi...
3,Content Writer,Writer,Young & Cerebral,Creative & Design,Lagos,Full Time,We are looking for a smart and intelligent Con...,HND,Entry level,2 years,"Write clear and concise copy for ads, scripts...",75000,"75,000 - 150,000",Yesterday,2023-11-02,2023-11-03 08:21:30,https://www.jobberman.com/listings/content-wri...
4,Procurement/Admin Personnel,Administrative Officer,StreSERT Integrated Limited,Admin & Office,Lagos,Full Time,The Procurement/ Admin personnel will be respo...,HND,Entry level,2 years,"Procurement: (office supplies, assets, station...",75000,"75,000 - 150,000",Yesterday,2023-11-02,2023-11-03 08:21:39,https://www.jobberman.com/listings/procurement...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Dental Nurse,Not given,Celeb Beauty Clinic,Medical & Pharmaceutical,Lagos,Full Time,We are looking to hire a suitable candidate to...,Degree,Entry level,1 year,"Prepare the dental surgery for each patient, e...",75000,"75,000 - 150,000",Yesterday,2023-11-02,2023-11-03 08:34:07,https://www.jobberman.com/listings/nurse-7wv2xm
184,Social Media Intern (NYSC),Not given,Young & Cerebral,Marketing & Communications,Lagos,Internship & Graduate,We are looking for an enthusiastic and smart i...,HND,No Experience,No Experience/Less than 1 year,Manage and maintain the organization's social ...,75000,"Less than 75,000",Yesterday,2023-11-02,2023-11-03 08:34:17,https://www.jobberman.com/listings/social-medi...
185,IT Officer,IT Officer,Jobberman (Third Party Recruitment),Engineering & Technology,Lagos,Full Time,We are looking for a qualified IT officer that...,Degree,Entry level,2 years,"Installing and configuring computer hardware, ...",Not given,Not given,Yesterday,2023-11-02,2023-11-03 08:34:27,https://www.jobberman.com/listings/it-officer-...
186,Accountant,Not given,Femstone Engineering Ltd,"Accounting, Auditing & Finance",Abuja,Full Time,We are seeking a skilled and experienced accou...,Degree,Entry level,1 year,"Perform day-to-day accounting activities, incl...",75000,"75,000 - 150,000",1 week ago,2023-10-27,2023-11-03 08:28:19,https://www.jobberman.com/listings/accountant-...


In [351]:
# # pd.read_csv("Completed_Cleaned_Jobberman.csv")
# small_df["Generic_Job_Name"].str.len().max()

In [323]:
# small_df["Generic_Job_Name"].apply(lambda x: x[0] if len(x)==1 else x)

0                                    Graphics Designer
1                     IT/Technical/Engineering Support
2                                                Other
3                                           Pharmacist
4                               Administrative Officer
5                                    Graphics Designer
6                                                Other
7                                 Sales Representative
8                                                Other
9                               Administrative Officer
10                                Sales Representative
11                                              Writer
12                                   Financial Officer
13                                   Financial Officer
14                          Customer Service & Support
15    Social Media Personnel/Marketer/Digital Marketer
16                      Hospitality/Culinary Personnel
17                             Digital Media Executive
18        

In [45]:
# chunk_size = 20  # Chunk size for splitting DataFrame
# if len(df) > chunk_size:
#     for i in range(0, len(df), chunk_size):
#         small_data = df.iloc[i:i + chunk_size, :]
#         print(i)
#         print(small_data)
#         small_data.to_csv(f"Jobberman_data_raw{i}.csv", index = False)
#         if small_data.empty:
#             print("empty")
#             break
#         begin = i
#         print("Cleaning and Transforming in chunks of 20")

0
                            Job_Name  \
0                   Factory Engineer   
1                    Proposal Writer   
2                 Factory Operations   
3                     Azure Engineer   
4     Microsoft 365 Support Engineer   
5     Regional Sales Advocate Member   
6           Regional Sales Member ll   
7            Regional Sales Member l   
8    Regional Sales Advocate Manager   
9             Regional Sales Manager   
10        Regional Marketing Manager   
11        Regional Promoters Manager   
12      Business Development Officer   
13                 Managing Director   
14                            Lawyer   
15    Account Client Service Officer   
16  Brand Promoter/Content Developer   
17               Executive Assistant   
18                   General Manager   
19                  Factory Engineer   

                                         Company_Name  \
0                       \n\nEmpire Energy Limited\n\n   
1                     \n\nSYNERGY INSIGHT L

Cleaning and Transforming in chunks of 20
440
                                         Job_Name  \
440  Marketing & Admission/Visa Assistant (MAVA).   
441                                    Accountant   
442                          Production Assistant   
443                                     Architect   
444                  Program & Evaluation Officer   
445                                UI/UX Designer   
446                         PHP Backend Developer   
447                              Chief Accountant   
448                        Communications Manager   
449            Local and Continental Cuisine Chef   
450             Programs and Partnerships Manager   
451                                     Secretary   
452                     Customer Relation Officer   
453                             Operation Manager   
454                                 Event Manager   
455                            Personal Assistant   
456    Regional Marketing Members - Entertainment   


In [ ]:
# df["Salary"] = df["Salary_Range"].fillna(0).apply(clean_salary)
# salary_rename(df)

In [ ]:
# from azure.identity import DefaultAzureCredential
# from azure.storage.blob import BlobServiceClient
# load_dotenv("azure_client_credentials.env")
# # Acquire a credential object
# token_credential = DefaultAzureCredential()

# blob_service_client = BlobServiceClient(
#         account_url=f"https://{storage_account_name}.blob.core.windows.net",
#         credential=token_credential)




# def upload_blob_file(blob_service_client: BlobServiceClient, container_name: str, local_file_path: str, blob_name: str):
#     # Get the container client
#     container_client = blob_service_client.get_container_client(container_name)

#     # Specify the path to the local file you want to upload
#     with open(local_file_path, "rb") as data:
#         # Upload the file to the container
#         blob_client = container_client.upload_blob(name=blob_name, data=data, overwrite=True)

#     # Return the URL of the uploaded blob
#     return blob_client.url


# curr_time = datetime.now()
# datetime_val = curr_time.strftime("%Y-%m-%d %H:%M:%S")

# upload_blob_file(blob_service_client, "cleaneddata", "Jobberman_data_raw.parquet", 
#                      f"Jobberman_data_raw_{datetime_val}.parquet")
    

In [257]:
# try:
#     completed_df = pd.read_csv("Completed_Cleaned_Jobberman.csv")
#     df = pd.read_csv("Jobberman_data_raw.csv")
#     if set(df["url"]) == set(completed_df["url"]):
#         print("Cleaning and transformation completed")

#     elif set(df["url"]) > set(completed_df["url"]):
#         remainder_df = df.loc[~df["url"].isin(list(completed_df["url"])), :]
#         remainder_df.to_csv("Jobberman_data_raw_remainder.csv", index = False)

#     else:
#         print("Already processed. File not cleaned up!")
#         remainder_df = completed_df.loc[~completed_df["url"].isin(list(df["url"])), :]
#         remainder_df.to_csv("Jobberman_data_raw_remainder.csv", index = False)


# except FileNotFoundError as e:
#     print(f"Completed Cleaned Jobberman csv file could not be found.\nPython error: {e}")
#     #print("Removing the raw CSV file now...")
#     #remove_raw_data_from_path("./Jobberman_data_raw.csv")#step 11


# try:
#     completed_df.to_parquet("Completed_Cleaned_Jobberman.parquet", index= False)

# except FileNotFoundError as e:
#     print(f"Completed Cleaned Jobberman csv file could not be found.\nPython error: {e}")

In [259]:
# completed_df

In [260]:
# remainder_df

In [ ]:
job_skills = {
    'social media personnel/marketer/digital marketer': [
        'Social media marketing',
        'Content creation',
        'SEO',
        'Branding',
        'Analytics',
        'Advertising',
        'Copywriting',
        'Influencer marketing',
        'Graphic design',
        'Email marketing',
        'Content strategy',
        'Social media ads',
        'Data analytics',
        'Marketing automation',
        'Online presence',
        'Social media trends',
        'Audience engagement',
        'Community management',
        'Campaign optimization',
        'ROI analysis',
    ],
    'sales representative': [
        'Sales',
        'Customer relationship',
        'Cold calling',
        'Negotiation',
        'Prospecting',
        'Closing deals',
        'Product knowledge',
        'Communication',
        'Territory management',
        'CRM',
        'Sales strategy',
        'Lead generation',
        'Client acquisition',
        'Client retention',
        'Sales forecasting',
        'Market research',
        'Competitive analysis',
        'Objection handling',
        'Sales presentations',
        'Sales quotas',
    ],
    'personal assistant/secretary/receptionist': [
        'Administrative support',
        'Scheduling',
        'Communication',
        'Office management',
        'Organization',
        'Customer service',
        'Filing',
        'Meeting coordination',
        'Data entry',
        'Multitasking',
        'Calendar management',
        'Time management',
        'Email correspondence',
        'Office software proficiency',
        'Telephone etiquette',
        'Discretion',
        'Confidentiality',
        'Document preparation',
        'Travel coordination',
        'Office supplies management',
    ],
    'financial officer': [
        'Financial analysis',
        'Accounting',
        'Budgeting',
        'Auditing',
        'Financial reporting',
        'Risk management',
        'Investment',
        'Taxation',
        'Finance software',
        'Compliance',
        'Financial modeling',
        'Cash flow analysis',
        'Asset management',
        'Financial planning',
        'Expense tracking',
        'Regulatory knowledge',
        'Market trends',
        'Cost control',
        'Mergers and acquisitions',
    ],
    'graphics designer': [
        'Graphic design',
        'Adobe Creative Suite',
        'Visual communication',
        'Typography',
        'Layout design',
        'Illustration',
        'Branding',
        'Creativity',
        'Color theory',
        'Image editing',
        'Web design',
        'Print design',
        'User interface design',
        'Animation',
        'Visual storytelling',
        'Photo manipulation',
        'Artistic flair',
        'Design software',
        'Client collaboration',
        'Attention to detail',
    ],
    'teacher': [
        'Classroom management',
        'Curriculum development',
        'Lesson planning',
        'Teaching methods',
        'Educational technology',
        'Assessment',
        'Communication',
        'Patience',
        'Adaptability',
        'Subject knowledge',
        'Student engagement',
        'Differentiated instruction',
        'Multicultural competence',
        'Classroom discipline',
        'Parent-teacher communication',
        'Assessment strategies',
        'Learning resources',
        'Interactive teaching',
        'Problem-solving',
        'Mentoring',
    ],
    'fashion designer (hair/clothes)': [
        'Fashion design',
        'Trend analysis',
        'Pattern making',
        'Sewing',
        'Fabric selection',
        'Creativity',
        'Color coordination',
        'Tailoring',
        'Styling',
        'Garment construction',
        'Textile knowledge',
        'Fashion illustration',
        'Client consultation',
        'Market research',
        'Sustainable fashion',
        'Fashion shows',
        'Prototyping',
        'Costume design',
        'Brand identity',
        'Merchandising',
    ],
    'logistics officer': [
        'Supply chain management',
        'Inventory control',
        'Transportation',
        'Warehouse management',
        'Logistics software',
        'Procurement',
        'Distribution',
        'Freight management',
        'Vendor relations',
        'Documentation',
        'Supply chain optimization',
        'Inventory tracking systems',
        'Demand forecasting',
        'Customs regulations',
        'Inventory audit',
        'Quality control',
        'Logistics analytics',
        'Strategic sourcing',
        'Inventory replenishment',
        'Reverse logistics',
    ],
    'other': [
        'Varied skills depending on the specific role',
    ],
    'lawyer': [
        'Legal research',
        'Litigation',
        'Contract law',
        'Legal writing',
        'Advocacy',
        'Negotiation',
        'Legal analysis',
        'Client representation',
        'Ethics',
        'Jurisprudence',
        'Case preparation',
        'Legal counseling',
        'Arbitration',
        'Mediation',
        'Legal strategy',
        'Courtroom skills',
        'Legal ethics',
        'Client confidentiality',
        'Legal code interpretation',
    ],
    'hospitality/culinary personnel': [
        'Culinary skills',
        'Customer service',
        'Food safety',
        'Menu planning',
        'Event management',
        'Hotel management',
        'Beverage service',
        'Culinary trends',
        'Hospitality software',
        'Food presentation',
        'Service etiquette',
        'Wine pairing',
        'Quality control',
        'Hygiene standards',
        'Menu development',
        'Customer satisfaction',
        'Restaurant management',
        'Guest experience',
        'Food and beverage knowledge',
    ],
    'doctor/nurse': [
        'Medical diagnosis',
        'Patient care',
        'Treatment',
        'Medical procedures',
        'Health assessment',
        'Disease prevention',
        'Medication administration',
        'Clinical skills',
        'Emergency response',
        'Compassion',
        'Critical thinking',
        'Patient education',
        'Healthcare regulations',
        'Clinical records',
        'Diagnostic tools',
        'Patient advocacy',
        'Infection control',
        'Surgical skills',
        'Medical ethics',
    ],
    'Inventory Manager/Store Keeper': [
        'Inventory management',
        'Stock control',
        'Record keeping',
        'Replenishment',
        'Warehousing',
        'Stocktaking',
        'Supply chain',
        'Organization',
        'Data analysis',
        'Stock rotation',
        'Shipment coordination',
        'Inventory audit',
        'Quality control',
        'Vendor relations',
        'Demand forecasting',
        'Inventory software',
        'Stock level optimization',
        'Cycle counting',
        'Cross-docking',
    ],
    'laboratory administrator': [
        'Lab equipment management',
        'Safety protocols',
        'Data recording',
        'Inventory control',
        'Procurement',
        'Quality assurance',
        'Lab report preparation',
        'Compliance',
        'Calibration',
        'Laboratory software',
        'Hazardous materials handling',
        'Experiment planning',
        'Equipment maintenance',
        'Chemical inventory',
        'Research support',
        'Sample handling',
        'Laboratory safety training',
        'Data analysis software',
        'Protocol adherence',
    ],
    'pharmacist': [
        'Medication dispensing',
        'Prescription evaluation',
        'Drug interactions',
        'Compounding',
        'Pharmacology',
        'Patient counseling',
        'Regulatory compliance',
        'Medication therapy management',
        'Drug information',
        'Health promotion',
        'Pharmacy software',
        'Prescription verification',
        'Inventory management',
        'Patient safety',
        'Medication dispensing systems',
        'Pharmaceutical sciences',
        'Prescription accuracy',
        'Pharmacy law',
        'Patient education',
        'Pharmacy operations',
    ],
    'business developer': [
        'Market analysis',
        'CRM (Customer Relationship Management)',
        'Business acumen',
        'Lead generation',
        'Sales forecasting',
        'Presentation skills',
        'Product knowledge',
        'Market research',
        'Competitive analysis',
        'Networking',
        'Strategic partnerships',
        'Customer acquisition',
        'Sales strategy',
        'Deal closing',
        'Client relationship management',
        'Sales negotiation',
        'Market trends',
        'Stakeholder engagement',
        'Sales analytics',
        'Business growth strategies',
    ],
    'inventory manager/store keeper': [
        'Supply chain coordination',
        'Inventory control systems',
        'Supplier negotiation',
        'Cycle counting',
        'Quality control',
        'JIT (Just-In-Time) inventory',
        'Inventory valuation',
        'ERP systems',
        'Cross-docking',
        'Stock level optimization',
        'Inventory software',
        'Inventory audits',
        'Procurement management',
        'Supply chain optimization',
        'Inventory tracking systems',
        'Vendor management',
        'Inventory reporting',
        'Stock replenishment',
        'Shipping logistics',
        'Warehouse organization',
    ],
    'banker': [
        'Financial advising',
        'Mortgage lending',
        'Risk assessment',
        'Investment management',
        'Credit analysis',
        'Regulatory compliance',
        'Financial products',
        'Wealth management',
        'Treasury services',
        'Customer relationship management',
        'Portfolio management',
        'Loan origination',
        'Financial planning',
        'Asset management',
        'Banking regulations',
        'Private banking',
        'Financial reporting',
        'Anti-money laundering',
        'Investment advisory',
        'Financial research',
    ],
    'it/technical/engineering support': [
        'Technical troubleshooting',
        'System administration',
        'Help desk support',
        'IT security protocols',
        'Hardware repair',
        'Network monitoring',
        'Software installation',
        'Technical documentation',
        'Operating systems',
        'Technical training',
        'Software development support',
        'IT infrastructure',
        'Troubleshooting software',
        'Network configurations',
        'IT security awareness',
        'Software upgrades',
        'User support',
        'Hardware maintenance',
        'Cloud computing',
        'Backup solutions',
    ],
    'Human Resource Officer/Hiring Manager': [
        'Recruitment strategies',
        'Compensation and benefits',
        'HR policies and procedures',
        'Employee relations',
        'Conflict resolution',
        'Talent acquisition',
        'Onboarding processes',
        'Diversity and inclusion initiatives',
        'Performance management',
        'Labor law knowledge',
        'HR software',
        'Workforce planning',
        'Workplace safety',
        'Employee development',
        'HR compliance',
        'HR analytics',
        'HR metrics',
        'Employment law',
        'Employee engagement',
        'Organizational development',
    ],
    'Administrative Officer': [
        'Office management',
        'Time management',
        'Documentation',
        'Data entry',
        'Records management',
        'Administrative software',
        'Customer service',
        'Meeting coordination',
        'Organization',
        'Communication',
        'Scheduling',
        'Bookkeeping',
        'Administrative support',
        'Resource management',
        'Office supplies',
        'Email management',
        'Task prioritization',
        'Meeting preparation',
        'Travel arrangements',
    ],
    'Intern': [
        'Learning and adapting',
        'On-the-job training',
        'Collaboration',
        'Work ethic',
        'Communication',
        'Professionalism',
        'Time management',
        'Research',
        'Networking',
        'Problem-solving',
        'Mentoring',
        'Teamwork',
        'Project support',
        'Data analysis',
        'Critical thinking',
        'Innovation',
        'Industry knowledge',
        'Adaptation to change',
        'Tech-savviness',
        'Creativity',
    ],
    'Housekeeper': [
        'Cleaning techniques',
        'Housekeeping standards',
        'Attention to detail',
        'Organization',
        'Time management',
        'Cleaning supplies management',
        'Guest services',
        'Laundry management',
        'Sanitation',
        'Communication',
        'Cleaning efficiency',
        'Inventory control',
        'Cleaning equipment',
        'Safety procedures',
        'Cleaning schedule',
        'Quality assurance',
        'Property maintenance',
        'Guest satisfaction',
        'Guest requests',
        'Discretion',
    ],
    'Architect/Engineer': [
        'Technical skills',
        'CAD software proficiency',
        'Blueprint reading',
        'Structural analysis',
        'Project management',
        'Sustainable design',
        'Building codes',
        'AutoCAD',
        'Construction management',
        'Site planning',
        'Design software',
        'Cost estimation',
        'Environmental sustainability',
        'Project scheduling',
        'Technical drawing',
        'Quality control',
        'Team collaboration',
        'Stakeholder communication',
        'Problem-solving',
        '3D modeling',
    ],
    'Beautician/Therapist': [
        'Beauty treatments',
        'Skincare',
        'Massage therapy',
        'Cosmetology',
        'Client consultation',
        'Product knowledge',
        'Spa services',
        'Wellness counseling',
        'Relaxation techniques',
        'Safety and hygiene',
        'Skin analysis',
        'Aromatherapy',
        'Beauty product trends',
        'Body treatments',
        'Holistic wellness',
        'Therapeutic techniques',
        'Client experience',
        'Health and wellness',
        'Salon management',
        'Body therapies',
    ],
    'Software Developer/Engineer': [
        'Programming languages',
        'Software development methodologies',
        'Debugging',
        'Algorithm design',
        'Problem-solving',
        'Software architecture',
        'Version control',
        'Database management',
        'Agile development',
        'Continuous integration',
        'System integration',
        'Testing frameworks',
        'Code optimization',
        'Software testing',
        'Quality assurance',
        'Front-end development',
        'Back-end development',
        'Full-stack development',
        'Machine learning',
        'Cybersecurity',
    ],
    'Vocational Worker': [
        'Specific skills',
        'Vocational training',
        'Trade skills',
        'Certifications',
        'Industry expertise',
        'Craftsmanship',
        'Technical proficiency',
        'Tool proficiency',
        'Safety practices',
        'Hands-on experience',
        'Quality workmanship',
        'Equipment operation',
        'Skill refinement',
        'Mastering the trade',
        'Building skills',
        'Technical competency',
        'Problem-solving',
        'Skilled labor',
        'Crafting expertise',
    ],
    'Data Expert': [
        'Data analysis',
        'Data management',
        'Data visualization',
        'Data mining',
        'Database administration',
        'Statistical analysis',
        'Data modeling',
        'Big data tools',
        'Data security',
        'Data-driven decision-making',
        'Data engineering',
        'Data transformation',
        'Data architecture',
        'Data integration',
        'Data governance',
        'Predictive analytics',
        'Data cleansing',
        'Data querying',
        'Machine learning',
        'Data ethics',
    ],
    'Scientist': [
        'Scientific research',
        'Experimentation',
        'Data analysis',
        'Hypothesis testing',
        'Laboratory techniques',
        'Research design',
        'Critical thinking',
        'Scientific method',
        'Data interpretation',
        'Report writing',
        'Experimental protocols',
        'Statistical methods',
        'Publication writing',
        'Data validation',
        'Quality control',
        'Data collection',
        'Fieldwork',
        'Lab safety',
        'Scientific publications',
        'Research collaboration',
    ],
    'IT Security Officer': [
        'Cybersecurity',
        'Network security',
        'Information security policies',
        'Risk assessment',
        'Security protocols',
        'Incident response',
        'Vulnerability assessment',
        'Security software',
        'Compliance',
        'Security awareness training',
        'Threat analysis',
        'Firewall management',
        'Intrusion detection',
        'Data encryption',
        'Security audits',
        'Forensic analysis',
        'Security architecture',
        'Identity and access management',
        'Penetration testing',
        'Security risk management',
    ],
    'Writer': [
        'Writing proficiency',
        'Creative writing',
        'Editing',
        'Research',
        'Content creation',
        'Copywriting',
        'Proofreading',
        'Adaptability',
        'SEO writing',
        'Communication',
        'Content strategy',
        'Content marketing',
        'Publication management',
        'Audience engagement',
        'Storytelling',
        'Narrative development',
        'Journalism',
        'Scriptwriting',
        'Technical writing',
        'Authorship',
    ],
    'Chief Operating Officer': [
        'Operations management',
        'Process improvement',
        'Efficiency optimization',
        'Strategic planning',
        'Leadership',
        'Budget management',
        'Risk assessment',
        'Supply chain management',
        'Team coordination',
        'Crisis management',
        'Change management',
        'Project management',
        'Stakeholder relations',
        'Performance metrics',
        'Decision-making',
        'Cross-functional teams',
        'Quality control',
        'Financial acumen',
        'Market analysis',
        'Legal compliance',
    ],
    'Stocks/FX Trader': [
        'Financial analysis',
        'Trading strategies',
        'Market research',
        'Risk management',
        'Investment analysis',
        'Technical analysis',
        'Trading platforms',
        'Trading psychology',
        'Financial regulations',
        'Algorithmic trading',
        'Investment strategies',
        'Derivatives trading',
        'Commodity trading',
        'Hedging',
        'Forex markets',
        'Risk assessment',
        'Market volatility',
        'Quantitative analysis',
        'Trading indicators',
        'Portfolio diversification',
    ],
    'Consultant': [
        'Problem-solving',
        'Analysis',
        'Client consulting',
        'Project management',
        'Business strategy',
        'Data analysis',
        'Presentation',
        'Industry knowledge',
        'Change management',
        'Communication',
        'Strategic planning',
        'Market entry strategies',
        'Consulting methodologies',
        'Market trends',
        'Stakeholder management',
        'Consultant-client relations',
        'Recommendation implementation',
        'Client satisfaction',
        'Benchmarking',
        'Advisory services',
    ],
    'Relationship Manager': [
        'Customer relationship management',
        'Client communication',
        'Account management',
        'Financial analysis',
        'Sales strategies',
        'Negotiation',
        'Problem-solving',
        'Cross-selling',
        'Client retention',
        'Product knowledge',
        'Sales relationship building',
        'Customer satisfaction',
        'Financial planning',
        'Client meetings',
        'Consultative selling',
        'Sales performance analysis',
        'Market insights',
        'Client portfolio management',
        'Sales presentations',
        'Strategic account management',
    ],
    'CEO/MD (Chief Executive Officer/Managing Director)': [
        'Leadership',
        'Strategic planning',
        'Decision-making',
        'Business vision',
        'Stakeholder management',
        'Corporate governance',
        'Financial acumen',
        'Team leadership',
        'Crisis management',
        'Innovation',
        'Organizational strategy',
        'Global business perspective',
        'Executive management',
        'Board relations',
        'Investor relations',
        'Corporate social responsibility',
        'Executive reporting',
        'Leadership development',
        'Industry thought leadership',
        'Legal compliance',
    ],
    'Product Management Officer': [
        'Product management',
        'Product development',
        'Market research',
        'Product lifecycle',
        'Product strategy',
        'Roadmapping',
        'Customer feedback',
        'Competitive analysis',
        'Product launch',
        'Product documentation',
        'Product innovation',
        'Product positioning',
        'Product marketing',
        'Product analytics',
        'Product design',
        'User experience (UX)',
        'Product monetization',
        'Feature prioritization',
        'Agile product development',
        'Product branding',
    ],
    'Hospitality/Culinary Personnel': [
        'Culinary arts',
        'Food safety',
        'Customer service',
        'Menu planning',
        'Event management',
        'Hotel management',
        'Beverage service',
        'Culinary trends',
        'Hospitality software',
        'Food presentation',
        'Service etiquette',
        'Wine pairing',
        'Quality control',
        'Hygiene standards',
        'Menu development',
        'Customer satisfaction',
        'Restaurant management',
        'Guest experience',
        'Food and beverage knowledge',
        'Cost control',
    ],
    'Architect/Engineer/Technician': [
        'Technical skills',
        'CAD software proficiency',
        'Blueprint reading',
        'Structural analysis',
        'Project management',
        'Sustainable design',
        'Building codes',
        'AutoCAD',
        'Construction management',
        'Site planning',
        'Design software',
        'Cost estimation',
        'Environmental sustainability',
        'Project scheduling',
        'Technical drawing',
        'Quality control',
        'Team collaboration',
        'Stakeholder communication',
        'Problem-solving',
        '3D modeling',
        'Technical specifications',
        'Engineering systems',
        'Regulatory compliance',
        'Fieldwork',
        'Laboratory testing',
        'Prototype development',
        'Maintenance procedures',
        'Technical troubleshooting',
        'Equipment calibration',
    ],
}

# You can access the job skills using their respective job roles as keys.


In [83]:
#adf = pd.read_csv("Completed_Cleaned_Jobberman.csv")
#adf.head()

,Job_Name,Generic_Job_Name,Company_Name,Industry,Location,Job_Type,Job_Summary,Education_Requirements,Experience_Level_Requirements,Experience_Length_Requirements,Job_Description,Salary,Salary_Range,Date_Posted,Day_Posted,Scrape_Time,url
0,Factory Engineer,['Architect/Engineer/Technician'],Empire Energy Limited,Engineering & Technology,Abuja,Full Time,We are looking to hire a suitable candidate to...,Degree,Entry level,2 years,Evaluating existing manufacturing processes an...,Not given,Not given,Yesterday,2023-10-25,2023-10-26 02:32:33,https://www.jobberman.com/listings/factory-eng...
1,Proposal Writer,['Other'],SYNERGY INSIGHT LIMITED,Consulting & Strategy,Lagos,Full Time,The primary purpose of this position is to ass...,Degree,Mid level,3 years,"Experience working at a program management, di...",75000,"75,000 - 150,000",Yesterday,2023-10-25,2023-10-26 02:32:39,https://www.jobberman.com/listings/proposal-wr...
2,Factory Operations,['Administrative Officer'],Empire Energy Limited,Engineering & Technology,Abuja,Full Time,We are looking to hire a suitable candidate to...,High School (S.S.C.E),Entry level,2 years,Following health and safety standards.Maintain...,Not given,Not given,Yesterday,2023-10-25,2023-10-26 02:32:45,https://www.jobberman.com/listings/factory-ope...
3,Azure Engineer,['IT/Technical/Engineering Support'],SYNERGY INSIGHT LIMITED,Software & Data,Lagos,Full Time,Azure Data Engineers are responsible for desig...,Degree,Senior level,3 years,Develop and implement data using Azure cloud m...,150000,"150,000 - 250,000",Yesterday,2023-10-25,2023-10-26 02:32:51,https://www.jobberman.com/listings/azure-engin...
4,Microsoft 365 Support Engineer,['IT/Technical/Engineering Support'],SYNERGY INSIGHT LIMITED,Engineering & Technology,Lagos,Full Time,We seek to hire a suitable candidate to fill t...,Degree,Mid level,3 years,Attend in-person meetings with clients to anal...,75000,"75,000 - 150,000",Yesterday,2023-10-25,2023-10-26 02:32:57,https://www.jobberman.com/listings/microsoft-3...


In [116]:
# #wrong_df = adf[adf["Generic_Job_Name"].str.startswith("[<OpenAIObject")]

# #string_data = wrong_df["Generic_Job_Name"][42]

# def correct_openai_format(string_data):
#     start_index = string_data.find('"content": "') + len('"content": "')
#     end_index = string_data.find('"', start_index)
#     content = string_data[start_index:end_index]
    
#     return content

# #     # Construct a list with the extracted content
# #     content_list = content
# #     print(content_list)

In [117]:
# pattern = r'<OpenAIObject.*? at.*?JSON' #r'[<OpenAIObject.*?>'
# import re

# np.where(adf["Generic_Job_Name"].str.startswith("[<OpenAIObject"), adf["Generic_Job_Name"].apply(correct_openai_format),
#         adf["Generic_Job_Name"])

array(["['Architect/Engineer/Technician']", "['Other']",
       "['Administrative Officer']",
       "['IT/Technical/Engineering Support']",
       "['IT/Technical/Engineering Support']", "['Sales Representative']",
       "['Sales Representative']", "['Sales Representative']",
       "['Sales representative']", "['Sales Representative']",
       "['Other']", "['Sales Representative']", "['Business developer']",
       "['CEO/MD (Chief Executive Officer/Managing Director)']",
       "['Lawyer']", "['Administrative Officer']", "['Other']",
       "['Administrative Officer']", "['Administrative Officer']",
       "['Architect/Engineer/Technician']", 'Generic_Job_Name',
       "['Architect/Engineer/Technician']", "['Writer']",
       "['Factory Operations']", "['IT/Technical/Engineering Support']",
       "['IT/Technical/Engineering Support']", "['Sales representative']",
       "['Sales Representative']", "['Sales representative']",
       "['Sales']", "['Sales representative']", "['Mark

In [ ]:
lowercase_dict = {}
for key, value in job_skills.items():
    lowercase_key = key.lower()
    if lowercase_key not in lowercase_dict:
        lowercase_dict[lowercase_key] = []
        lowercase_dict[lowercase_key].extend(value)

# Display the resulting dictionary
print(lowercase_dict)

In [ ]:
lowercase_dict.keys()

In [ ]:
job_skills.keys()

In [ ]:

# Specify the name of your JSON file
json_filename = "job_skills.json"

# Get a list of files in the current directory
current_directory = os.getcwd()
files_in_directory = os.listdir(current_directory)

# Check if your JSON file is in the list of files
if json_filename in files_in_directory:
    print(f"{json_filename} exists in the current directory.")
else:
    print(f"{json_filename} does not exist in the current directory.")
    with open("job_skills.json", "w") as outfile: 
        json.dump(lowercase_dict, outfile)
        
    print(f"{json_filename} created")


# Data to be written 
